In [1]:
import numpy as np
import pandas as pd

In [2]:
clicks = np.load("../preprocessing/clicks/clicks.npy")


labels_df = pd.read_csv("../preprocessing/labels.csv")

genres_dict = {"classical" : 0, "electronic" : 1, "pop" : 2, "rock" : 3} 

labels_df = labels_df["genre"]
labels_df = labels_df.map(genres_dict)

# remove pop and rock
idx_pop = labels_df.index[labels_df == 2].tolist()
idx_rock = labels_df.index[labels_df == 3].tolist()
idx_pop_rock = idx_pop + idx_rock


labels_df = labels_df.drop(idx_pop_rock)
print("Shape clicks before deleting poprock", np.shape(clicks))
clicks = np.delete(clicks, idx_pop_rock, axis=0)
print("Shape clicks after deleting poprock", np.shape(clicks))
clicks = clicks[:,:300]
print("Shape clicks after slicing", np.shape(clicks))

# shuffle
assert len(clicks) == len(labels_df), "Something is wrong"
n_data = len(clicks)

idx = np.random.permutation(n_data)
features, labels = clicks[idx], labels_df.reindex(idx)


Shape clicks before deleting poprock (400, 600)
Shape clicks after deleting poprock (200, 600)
Shape clicks after slicing (200, 300)


In [3]:
print(labels_df)

0      0
1      0
2      0
3      0
4      0
      ..
195    1
196    1
197    1
198    1
199    1
Name: genre, Length: 200, dtype: int64


In [6]:
# k-fold cross-validation

from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix
from sklearn import svm

k = 5
cv = KFold(n_splits=k)

kfold_train_metrics = []
kfold_test_metrics = []

kfold_train_cm = []
kfold_test_cm = []

total_input = clicks
total_labels = labels_df

cv = KFold(n_splits=k)

try: 
    if self.name == "LDA" or self.name == "QDA": 
        self._fit()
except: pass

for train_index, test_index in cv.split(total_input):

    train_features, train_labels = total_input[train_index], total_labels.iloc[train_index]
    test_features, test_labels = total_input[test_index], total_labels.iloc[test_index]

    clf = svm.SVC(kernel='linear')
    clf.fit(train_features, train_labels)

    kfold_train_metrics.append(clf.score(train_features, train_labels))
    kfold_test_metrics.append(clf.score(test_features, test_labels))

    kfold_train_cm.append(confusion_matrix(train_features, train_labels))
    kfold_test_cm.append(confusion_matrix(test_features, test_labels))

mean_acc = str(100*np.mean(kfold_test_metrics))[:5]
std_acc = str(100*np.std(kfold_test_metrics))[:4]
print(f"Mean accuracy: {mean_acc} ± {std_acc} %")

ValueError: Classification metrics can't handle a mix of multilabel-indicator and binary targets